In [1]:
from mxnet import gluon
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn 
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import PIL.Image

from gluoncv.data import VOCSegmentation
from gluoncv.utils.metrics.voc_segmentation import batch_pix_accuracy, batch_intersection_union

%matplotlib inline

/home/jiancheng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
import gluoncv

In [3]:
gluoncv.__version__

'0.3.0'

In [4]:
from mylib.deeplabv3p import DeepLabv3p

In [5]:
model = DeepLabv3p(OS=8)
model.initialize(ctx=mx.gpu())
weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
# weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [6]:
# ??transforms

In [8]:
#     input_transform = transforms.Compose([
#         lambda x: nd.array(np.array(x)),
#         transforms.ToTensor(),
# #         transforms.Normalize([.485, .456, .406], [.229, .224, .225]),
#     ])
# input_transform = lambda x: nd.array(np.array(x.resize((512,512))).transpose(2,0,1))/127.5 - 1.

In [9]:
class CustomVOCSegmentation(VOCSegmentation):
    
    def __init__(self, split, size=(512,512)):
        super().__init__(split=split)
        self.size = size
        
    def __getitem__(self, index):
        img = PIL.Image.open(self.images[index]).convert('RGB')
        mask = PIL.Image.open(self.masks[index])
        img = img.resize(self.size, PIL.Image.BILINEAR)
        mask = mask.resize(self.size, PIL.Image.NEAREST)
        img = self._img_transform(img).transpose((2,0,1))/127.5 - 1.
        mask = self._mask_transform(mask)
        return img, mask

In [10]:
dataset = CustomVOCSegmentation(split='val')

In [11]:
img, seg = dataset[36]

In [12]:
img.shape

(3, 512, 512)

In [13]:
seg.shape

(512, 512)

In [14]:
img.max()


[ 0.92156863]
<NDArray 1 @cpu(0)>

In [15]:
img.min()


[-1.]
<NDArray 1 @cpu(0)>

In [16]:
img.size

786432

In [17]:
seg.max()


[ 1.]
<NDArray 1 @cpu(0)>

In [18]:
dataloader = gluon.data.DataLoader(dataset, batch_size=8)

In [20]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
#     x = (x.transpose((0,3,1,2)) / 127.5 - 1.).copyto(mx.gpu())
    x = x.copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
#     print(x.shape, y.shape, pred.shape)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
#     print(pix_acc.shape, IoU.shape)
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))
#     break

iter181: pix_acc: 0.9677, mIoU: 0.8435: 100%|██████████| 182/182 [05:31<00:00,  1.82s/it]


In [21]:
model = DeepLabv3p(OS=8)
model.initialize(ctx=mx.gpu())
weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
# weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [22]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = (x.transpose((0,3,1,2)) / 127.5 - 1.).copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
#     print(x.shape, y.shape, pred.shape)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
#     print(pix_acc.shape, IoU.shape)
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))
#     break

  0%|          | 0/182 [00:00<?, ?it/s]

MXNetError: Shape inconsistent, Provided = [32,3,3,3], inferred shape=(32,512,3,3)

In [ ]:
model = DeepLabv3p(OS=16)
model.initialize(ctx=mx.gpu())
# weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [ ]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = (x.transpose((0,3,1,2)) / 127.5 - 1.).copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
#     print(x.shape, y.shape, pred.shape)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
#     print(pix_acc.shape, IoU.shape)
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))

In [ ]:
IoU.mean()

In [ ]:
np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)

In [ ]:
mIoU

In [1]:
from torchvision.utils import make_grid

In [2]:

?make_grid